Paired river network conflation

Please reference this publication whenever this code is used
http://geomorphometry.org/system/files/Read2011geomorphometry.pdf
Thanks

Takes about 25 minutes to read in the entire geofabric.
Takes less then 5 minutes to run the analysis for Tasmania (17k stream links).


Is not memory /time efficent yet when scaling to the entire geofabric dataset (1.5 millon stream links)

In [38]:
netGDB = r'C:\data\temp\SH_Network_GDB_V2_1_1\SH_Network_GDB\SH_Network.gdb'
pkl_path = r'C:\data\temp'
#checks all nodes within at least this distance for a conflation
#its a time saving optimisation and should be set on the large size to avoid missing the best result
#A guide to setting this is: set it a bit bigger then the largest expected separation of conflated nodes
#for small networks networks just set it very large
#in projection units (1degree approx 100km)
searchRadius = 0.1
#number of potential matches to keep for each feature
maxMatchKeep = 10

In [93]:
from shapely.prepared import prep
from shapely.geometry import shape
from shapely.geometry import mapping,Polygon,Point,LineString
import fiona
from rtree import index
import networkx as nx
import collections
import os

first step is to produce a networkx multiDiGraph from your source data.

The code below does that specifically for the Australian geofabric dataset in ESRI geodatabase format

(note the geodatabase has to be upgraded in ESRI software first so that gdal can read it).

For other datasets it would need to be changed to produce the same output from the different inputs.

This needs to be done for both the conflation source and destination networks

the key thing is some sort of from and to node attribution to build the network structure between subcatchments. if this isnt available it can be generated from the coords of the stream links. Code for this not here but available on request.

subcatchments and streams need a one to one relationship. streams without a catchment are (i think) fully handled. catchments without streams are ignored.

In [14]:
def remove_geofabric_catch_duplicates(DG):
    '''deal with special case of duplicate catchment polygons in the geofabric
    
    Im not sure why the geofabric allows multiple stream features to 
    have a relationship to the same subcatchment.
    Maybe it is ok and i just need to change my expected data model..... will have to think about that.
    Its a bit inconvinient though. 
    Is there a good way to weed these out logically and consistently?
    Here is my attempt. Based on the fact that they seem to occur only at flow splits.
    '''
    ts = nx.topological_sort(DG)
    for n in ts:
        new_set = set()
        for f,t,k,data in DG.in_edges_iter(n,data=True,keys=True):
            new_set.update([(data['cid'])])
        for f,t,k,data in DG.out_edges_iter(n,data=True,keys=True):
            if data['cid'] in new_set:
                data['cid'] = None
                data['subCatch'] = Polygon()
    
    #unfortunatly many cases still remain
    #simple way of handling them is to make an arbitary choice
    #first in keeps the catchment
    #not ideal
    ss = set()
    for f,t,k,data in DG.edges_iter(data=True,keys=True):
        if data['cid'] is not None:
            if data['cid'] in ss:
                print 'WARNING: duplicate catchment removed' + str(data['cid'])
                data['cid'] = None
                data['subCatch'] = Polygon()
            #assert data['cid'] not in ss, 'duplicate catchment id ' + str(data['cid'])
            ss.add(data['cid'])

def read_geofabric_data(netGDB):
    catch = {}
    with fiona.open(netGDB, layer='AHGFCatchment') as c:
        for feat in c:
            geom = shape(feat['geometry'])
            cid = feat['properties']['HydroID']
            assert cid not in catch #shouldnt be duplicates 
            catch[cid] = geom
    
    DG=nx.MultiDiGraph()
    with fiona.open(netGDB, layer='AHGFNetworkStream') as c:
        for feat in c:
            streamLink = shape(feat['geometry'])
             #for some reason these are coming in as multipart features with only one part - no need for this
            assert streamLink.type == 'MultiLineString'
            assert len(streamLink.geoms) == 1
            streamLink = streamLink.geoms[0]
            
            ##remove this - just here for testing
            if streamLink.representative_point().y > -40.5: #tasmania for testing
                continue
            
            sid = feat['properties']['HydroID']
            cid = feat['properties']['DrainID']
            fid = feat['properties']['From_Node']
            tid = feat['properties']['To_Node']
            subCatch = catch.get(cid,Polygon())
            DG.add_edge(fid, tid, id=sid,cid=cid,subCatch=subCatch,stream=streamLink)
            
    return DG

In [6]:
DG1 = read_geofabric_data(netGDB)

In [15]:
remove_geofabric_catch_duplicates(DG1)

In [16]:
nx.write_gpickle(DG1, os.path.join(pkl_path, 'tas_conflation.p'))

For testing im just working with one network and shifting it spatially a bit.

it wont show the full capabilities of the tool but it is a useful test case.


In [101]:
#TESTING ONLY now shift the geometries a bit in the copy just to make it a useful test
DG2 = DG1.copy()
#assumes geographic coords
from shapely.ops import transform
for f,t,k,data in DG2.edges_iter(data=True,keys=True):
    data['subCatch'] = transform(lambda x, y, z=None: (x+0.01, y+0.01), data['subCatch'])
    data['stream'] =  transform(lambda x, y, z=None: (x+0.01, y+0.01), data['stream'])


In [ ]:
nx.write_gpickle(DG2, os.path.join(pkl_path, 'tas_conflation2.p'))

you can start from here by loading in the pickles that were created with the above code earlier.
Run the imports and global variables code at the top first though

In [102]:
def build_index(DG):
    '''build spatial and other indexes'''
    #remember rtrees are not picklable - doh
    DG_idx = index.Index()
    for f,t,k,data in DG.edges_iter(data=True,keys=True):
        if not data['subCatch'].is_empty:
            DG_idx.insert(0, data['subCatch'].bounds,obj=(f,t,k))
    return DG_idx

In [103]:
def upstream_edge_set(DG):
    '''build up a list of upstream edge ids in the destination network'''
    #wonder it if these lists become unwieldy for edges at the root of a big tree
    ts = nx.topological_sort(DG)
    for n in ts:
        new_set = set()
        for f,t,k,data in DG.in_edges_iter(n,data=True,keys=True):
            new_set.update((data['ids']))
        for f,t,k,data in DG.out_edges_iter(n,data=True,keys=True):
            data['ids'] = new_set.union([(f,t,k)])
            

In [104]:
def catch_area(DG):
    '''calc catchment area
    
    works with anabranching networks without double counting'''
    for f,t,k,data in DG.edges_iter(data=True,keys=True):
        catchArea = 0.0
        for e in data['ids']:
            catchArea += DG.get_edge_data(*e)['subCatch'].area
        data['catchArea'] = catchArea

In [105]:
def build_overlaps(DG1,DG2,DG2_idx):
    '''build up dictionary of overlapping areas between 2 graphs'''

    for f,t,data in DG1.edges_iter(data=True):
        geom = data['subCatch']
        data['overlaps'] = {}
        if not geom.is_empty:
            prepGeom = prep(geom)
            for e in DG2_idx.intersection(geom.bounds,objects='raw'):
                nGeom = DG2.get_edge_data(*e)['subCatch']
                if prepGeom.intersects(nGeom):
                    area = geom.intersection(nGeom).area
                    if area > 0.0:
                        data['overlaps'][e] = area


In [106]:
def build_full_overlaps(DG):
    '''build up table of overlapping area for the entire catchment above and including it

    assumes build_overlaps has been run with DG as the first network
    works with anabranching networks without double counting
    '''

    for f,t,k,data in DG.edges_iter(data=True,keys=True):
        fullOverlaps = collections.defaultdict(float)
        for e in data['ids']:
            for k, v in DG.get_edge_data(*e)['overlaps'].iteritems():
                fullOverlaps[k] += v
        data['fullOverlaps'] = fullOverlaps


In [45]:
#NOTE:upstream_edge_set,catch_area,build_overlaps, and build_full_overlaps create and store a lot of data
#for DG1 these could be chained together as the outer loop of find_all_matches to avoid keeping and of this intermediate data
#big saving on memory footprint possible for large DG1 networks.

Ok lets run some code

In [100]:
DG1 = nx.read_gpickle(os.path.join(pkl_path, 'tas_conflation.p'))

In [47]:
DG2 = nx.read_gpickle(os.path.join(pkl_path, 'tas_conflation2.p'))

In [107]:
DG2_idx = build_index(DG2)

In [108]:
upstream_edge_set(DG1)

In [109]:
upstream_edge_set(DG2)

In [110]:
catch_area(DG1)

In [111]:
catch_area(DG2)

In [112]:
build_overlaps(DG1,DG2,DG2_idx)

In [113]:
build_full_overlaps(DG1)

The next step is to find sum up areas to find the catchment overlap for every combination. We are only interested in the best or a short list of the overlaps that match well
The simple approach is a brute force exhustive test of all combinations. This works well for a few thousand (75 minutes for 17k x 17k) sub catchments in each graph. This would not scale well as network sizes increase.

There are a few ways to reduce the set of catchments to test for a match. One issue to keep in mind is to not make assumptions about how similar the two networks topology might be. This makes learning from nearby matches problematic although possible. limiting the search based on area similarity or stopping it based on an analysis of if any other catchments are likely to be a match is also possible.

However, To keep the code as simple and readable these were avoided in favour of a technique that was simple and effective. The approach taken is to use a tunable spatial proximity limit that should be set to a size that is expected to ensure finding the best matches within that radius. setting it too small would cause missed matches, too large would just take longer. This is the only downside of this method. I would have prefered to avoided confusing users with parameters to tune. Generally i would suggest setting if fairly large (10km?) to avoid issues but still get a reasonable speed.


In [114]:
def find_all_matches(DG1,DG2,DG2_idx,searchRadius,maxMatchKeep=1):
    '''for each catchment in DG1 find list the best matches
    
    This is a good and fairly efficent way of doing this.
    In a large network if you are only interested in a few catchment
    then you could write a similar routine that only ran for those catchments to save some time.
    Here the search is limited to nearby catchments by searchRadius
    This could easily be modified to limit in different ways
    for instance first order stream have no need to be searched beyond their own bounding box
    
    maxMatchKeep
    limits the number of potential matches that are kept to a short list of n items.
    The best items are chosen using a simple test of the quality of the match
    This is used to reduce the memory footprint of the returned dictionary
    '''
    matches = {}
    for f,t,k,data in DG1.edges_iter(data=True,keys=True):
        if len(data['fullOverlaps']) == 0:
            continue
        match = []
        searchBounds = Point(data['stream'].coords[-1]).buffer(searchRadius).bounds
        for e in DG2_idx.intersection(searchBounds,objects='raw'):
            data2 = DG2.get_edge_data(*e)
            overlap = sum( data['fullOverlaps'].get(k,0) for k in data2['ids'])
            if overlap > 0.0:
                qual = (overlap+overlap)/(data['catchArea']+data2['catchArea'])
                match.append((qual,overlap,e))
        #just keep the best few matches
        match.sort(reverse=True)
        matches[(f,t,k)] = match[0:maxMatchKeep]
    return matches

In [115]:
matches = find_all_matches(DG1,DG2,DG2_idx,searchRadius,maxMatchKeep)

In this process the features being matched together are the ends of a subcatchment. Just before a confluence, or the next reach. Nodes, if you like, but considering each inflow to a node seperatly.

Once you have this shortlist of matches for each feature you need to consider way to summarise that data.


The most basic option is to just take the best match for each feature. This gives you a node to node relationship (although considering each inflow to a node seperatly).

A useful way to build on that is to consider each incoming branch at a confluence together so that these confluences are accuratly identified. Wtihout this small differences in the very end of a catchment can see the best match shifting upstream a stream link or 2 to minimise the errors. This would give you a true node to node relationship.

Some uses might want to draw on additional attribute data to pick from the best matches. If this is required then in is probably because the subcatchment of the feature being conflated isnt significant enough to be well captured accuratly by one of the dems. nevertheless this is still possible but is left as additional work for the end user. The table of possible matches is available for this analysis.

Once a set of 'best' matches for points in the network has been selected these can then be expanded out to provide a conflation of reaches and subcatchments. 

You need to consider that the level of detail between the 2 networks may be very different and the topology may be very different.Any anabranching in the network also needs careful consideration. The differences can result is multiple features conflating to the same point, confluences being seperated or in a different order. These are generally going to be differences in the 2 networks not issues with the results from this tool. Ideally this tool can help highlight these differences.

Using these tools you are able to transfer attribute data between networks. Additionally you can start to make assessments of the level of detail that is supported by the DEM and the comparative qualities of the networks and the source DEMs.

Below are some simple tools for doing some of what has been described above. It is by no means an exhaustive set of tools and is subject to ongoing work.



In [116]:
best = [(k,l[0][2],l[0][0]) for k,l in matches.iteritems()]

In [117]:
def write_debug_lines(DG1,DG2,best,fileName):
    '''a simple output to show how each to node matches up
    
    handy for looking for errors in the conflation'''
    schema = {
        'geometry': 'LineString',
        'properties': {'qual': 'float'},
    }
    with fiona.open(fileName, 'w', 'ESRI Shapefile', schema) as c:
        for e1,e2,qual in best:
            p1 = DG1.get_edge_data(*e1)['stream'].coords[-1]
            p2 = DG2.get_edge_data(*e2)['stream'].coords[-1]
            geom = LineString(LineString([p1, p2]))
            c.write({
                'geometry': mapping(geom),
                'properties': {'qual': qual},
            })

In [118]:
write_debug_lines(DG1,DG2,best,os.path.join(pkl_path, 'debug_lines.shp'))

Note: not finished the reimplementation

TODO

unify the matches at a confluence to one match. difference between these and subcatchment match is also useful.

transfer/rewrite code that maps the conflation out along reaches and subcatchments

transfer/rewrite code that highlights topology differences




In [119]:
def write_catch(DG,fileName):
    schema = {
        'geometry': 'Polygon',
        'properties': {'id': 'int'},
    }
    with fiona.open(fileName, 'w', 'ESRI Shapefile', schema) as c:
        for f,t,data in DG.edges_iter(data=True):
            if not data['subCatch'].is_empty:
                c.write({
                    'geometry': mapping(data['subCatch']),
                    'properties': {'id': data['id']},
                })

def write_stream(DG,fileName):
    schema = {
        'geometry': 'LineString',
        'properties': {'id': 'int'},
    }
    with fiona.open(fileName, 'w', 'ESRI Shapefile', schema) as c:
        for f,t,data in DG.edges_iter(data=True):
            if not data['stream'].is_empty:
                c.write({
                    'geometry': mapping(data['stream']),
                    'properties': {'id': data['id']},
                })


In [120]:
write_catch(DG1,os.path.join(pkl_path, 'catch1.shp'))
write_catch(DG2,os.path.join(pkl_path, 'catch2.shp'))
write_stream(DG1,os.path.join(pkl_path, 'stream1.shp'))
write_stream(DG2,os.path.join(pkl_path, 'stream2.shp'))

In [ ]:
#some code used for error testing in development
#assumes full overlap
#will get some show up due to floating point issues
def test_stuff(DG):
    for f,t,data in DG.edges_iter(data=True):
        area = data['subCatch'].area
        area2 = 0.0
        for k, v in data['overlaps'].iteritems():
                area2 += v
        if (abs(area - area2) > 0.000000000000000001):
            print area,area2

    #a test for errors
    for f,t,data in DG.edges_iter(data=True):
        area = data['catchArea']
        area2 = 0.0
        for k, v in data['fullOverlaps'].iteritems():
                area2 += v
        if (abs(area - area2) > 0.00000000000001):
            print area,area2